In [ ]:
import cv2
import os
import numpy as np
import json

In [ ]:
class StreamStats(object):
    """ See https://www.johndcook.com/blog/standard_deviation/
    """
    def __init__(self):
        self.M = 0
        self.S = 0
        self.k = 0
        
    def update(self, x):
        self.k += 1
        if self.k == 1:
            self.M = x
            self.S = 0
        else:
            prevM = self.M
            prevS = self.S
            self.M = prevM + (x - prevM)/self.k
            self.S = prevS + (x - prevM) * (x - self.M)
        
    def mean(self):
        return self.M
        
    def variance(self):
        if self.k - 1 > 0:
            return self.S / (self.k - 1)
        else:
            return 0
    
    def std(self):
        return np.sqrt(self.variance())

In [ ]:
train_dir = "/home/lowik/sealion/data/sealion/TrainBlock/"

In [ ]:
stats = StreamStats()
block_status = dict()
# if more than 60% of the pixel are black, we won't use the block.
threshold_block_available = 0.6
for filename in sorted(os.listdir(train_dir))[n-1:]:
    im = cv2.imread(os.path.join(train_dir, filename))
    train_id, ext = filename.split("_")
    block_id, _ = ext.split(".")
    train_id, block_id = int(train_id), int(block_id)
    
    if train_id not in block_status:
        block_status[train_id] = np.zeros((16, ), np.bool)
        
    total_pixel = im.shape[0] * im.shape[1]
    if np.count_nonzero(im == 0)/total_pixel < threshold_block_available:
        block_status[train_id][block_id] = True
        
    mean_per_channel = np.mean(np.mean(im, axis=0), axis=0)
    stats.update(mean_per_channel)

print("Mean: ", stats.mean())
print("Std: ", stats.std())

In [ ]:
for tid in block_status.keys():
    # Need to convert to bool, np.bool is not JSON serializable.
    block_status[tid] = [bool(s) for s in block_status[tid]]

with open("/home/lowik/sealion/data/sealion/blocks_status.json", "w") as jfile:
    json.dump(block_status, jfile, indent=2)

In [ ]:
np.save("../data/sealion/mean_bgr.npy", stats.mean())
np.save("../data/sealion/std_bgr.npy", stats.std())